<a href="https://colab.research.google.com/github/sumit2312/Schedule/blob/master/Mask_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2,os
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import  Dense, Activation, Dropout, Conv2D, Flatten, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from keras.models import load_model

In [ ]:
#use the file path where your dataset is stored
data_path = r'/content/drive/MyDrive/face-mask-dataset/Dataset/train'
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]
label_dict = dict(zip(categories,labels))
print(label_dict)
print(categories)
print(labels)

{'without_mask': 0, 'with_mask': 1}
['without_mask', 'with_mask']
[0, 1]


In [ ]:
img_size = 150
data = []
target = []
for category in categories:
    folder_path = os.path.join(data_path,category) 
    img_names = os.listdir(folder_path)
    
    for img_name in img_names:
        img_path = os.path.join(folder_path,img_name)
        img = cv2.imread(img_path)
        try:
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray,(img_size,img_size))
            data.append(resized)
            target.append(label_dict[category])
        
        except Exception as e:
            print("Exception: ",e)

In [ ]:
data = np.array(data)/255.0  #data values are normalized
#reshaping of data                                                
data = np.reshape(data,(data.shape[0],img_size,img_size,1))
target = np.array(target)
new_target = np_utils.to_categorical(target)
#saving the files                                
np.save('data',data)
np.save('target',new_target)

In [ ]:
data = np.load('/content/data.npy')
target = np.load('/content/target.npy')
model = Sequential()
model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50,activation='relu'))
model.add(Dense(2,activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 200)     2000      
_________________________________________________________________
activation (Activation)      (None, 148, 148, 200)     0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 200)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 100)       180100    
_________________________________________________________________
activation_1 (Activation)    (None, 72, 72, 100)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 100)       0         
_________________________________________________________________
flatten (Flatten)            (None, 129600)            0

In [ ]:
train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=0.1)

In [ ]:
checkpoint=ModelCheckpoint('model-{epoch:03d}.model', monitor='val_loss', verbose = 0, save_best_only = True,mode='auto')


In [ ]:
history = model.fit(train_data,train_target,epochs = 5, callbacks = [checkpoint], validation_split = 0.2)

Epoch 1/5
30/30 [==============================] - 177s 6s/step - loss: 1.1813 - acc: 0.5289 - val_loss: 0.6987 - val_acc: 0.4958
INFO:tensorflow:Assets written to: model-001.model/assets
Epoch 2/5
30/30 [==============================] - 174s 6s/step - loss: 0.6515 - acc: 0.5842 - val_loss: 0.4021 - val_acc: 0.8655
INFO:tensorflow:Assets written to: model-002.model/assets
Epoch 3/5
30/30 [==============================] - 172s 6s/step - loss: 0.4173 - acc: 0.7990 - val_loss: 0.3085 - val_acc: 0.8908
INFO:tensorflow:Assets written to: model-003.model/assets
Epoch 4/5
30/30 [==============================] - 176s 6s/step - loss: 0.2852 - acc: 0.8944 - val_loss: 0.3068 - val_acc: 0.8655
INFO:tensorflow:Assets written to: model-004.model/assets
Epoch 5/5
30/30 [==============================] - 172s 6s/step - loss: 0.2785 - acc: 0.8846 - val_loss: 0.2008 - val_acc: 0.9496
INFO:tensorflow:Assets written to: model-005.model/assets


In [ ]:
plt.plot(history.history['acc'],'r',label='training accuracy')
plt.plot(history.history['val_acc'],'b',label='validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()
plt.plot(history.history['loss'],'r',label='training loss')
plt.plot(history.history['val_loss'],'b',label='validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

NameError: ignored

In [ ]:
print(model.evaluate(test_data,test_target))

5/5 [==============================] - 6s 1s/step - loss: 0.2105 - acc: 0.9248
[0.21047890186309814, 0.9248120188713074]


In [ ]:
from keras.models import load_model

model.save('./model-010.h5')  # creates a HDF5 file 'my_model.h5'


In [ ]:
Y_val_pred = model.predict(test_data)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(np.argmax(test_target, axis=1), np.argmax(Y_val_pred, axis=1))

0.924812030075188

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix with testing data')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=55)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
cm = confusion_matrix(np.argmax(test_target, axis=1), np.argmax(Y_val_pred, axis=1))

cm_plot_label =['mask', 'not mask']
plot_confusion_matrix(cm, cm_plot_label, title ='Confusion Metrix for Breast Cancer')

NameError: ignored